In [31]:
import sys
import os
from pathlib import Path
project_root = os.path.join('/home/shashank/research/hyper/HyperPPO')
os.chdir(project_root)
%pwd # should be PPGA root dir

from IPython.display import HTML, Image
from IPython.display import display
from brax.io import html, image


from sample_factory.enjoy import *
from sf_examples.brax.train_brax import register_brax_custom_components, parse_sf_args, add_extra_params_func, override_default_params_func, parse_full_cfg
from sample_factory.utils.wandb_utils import init_wandb
from tensorboardX import SummaryWriter

In [27]:
env = "humanoid"
hyper = "True"
train_dir = "train_dir/hyper/hyper_"
experiment = "08_hyper_see_8561347_env_humanoid_d.cri_False_m.std_True_a.s.mod_biased"
# milestone_name = "checkpoint_000198558_152616960.pth"
milestone_name = None

In [28]:
def parse_brax_cfg(eval_str, evaluation=False):
    parser, partial_cfg = parse_sf_args(evaluation=evaluation, argv=eval_str)
    add_extra_params_func(parser)
    override_default_params_func(partial_cfg.env, parser)
    final_cfg = parse_full_cfg(parser,eval_str)
    return final_cfg

In [29]:
eval_str = "python -m sf_examples.brax.enjoy_hyper_brax"
eval_str += f" --env {env}"
eval_str += f" --train_dir {train_dir}"
eval_str += f" --experiment {experiment}"

argv = eval_str.split()[3:]

register_brax_custom_components(evaluation=False)
cfg = parse_brax_cfg(evaluation=True, eval_str=argv)


[2023-08-23 18:07:49,948][17523] Environment ant already registered, overwriting...
[2023-08-23 18:07:49,949][17523] Environment humanoid already registered, overwriting...
[2023-08-23 18:07:49,950][17523] Environment halfcheetah already registered, overwriting...
[2023-08-23 18:07:49,951][17523] Environment walker2d already registered, overwriting...


In [30]:
verbose = True


cfg = load_from_checkpoint(cfg)

eval_env_frameskip: int = cfg.env_frameskip if cfg.eval_env_frameskip is None else cfg.eval_env_frameskip
assert (
    cfg.env_frameskip % eval_env_frameskip == 0
), f"{cfg.env_frameskip=} mmilestoneust be divisible by {eval_env_frameskip=}"
render_action_repeat: int = cfg.env_frameskip // eval_env_frameskip
cfg.env_frameskip = cfg.eval_env_frameskip = eval_env_frameskip
log.debug(f"Using frameskip {cfg.env_frameskip} and {render_action_repeat=} for evaluation")

cfg.num_envs = 1
cfg.env_agents = 1
cfg.quads_render = True
cfg.max_num_episodes = 4
cfg.eval_deterministic = True

render_mode = "human"
# render_mode = None
if cfg.save_video:
    render_mode = "rgb_array"
elif cfg.no_render:
    render_mode = None

env = make_env_func_batched(
    cfg, env_config=AttrDict(worker_index=0, vector_index=0, env_id=0), render_mode=render_mode
)
env_info = extract_env_info(env, cfg)

if hasattr(env.unwrapped, "reset_on_init"):
    # reset call ruins the demo recording for VizDoom
    env.unwrapped.reset_on_init = False


[2023-08-23 18:08:03,028][17523] Loading existing experiment configuration from train_dir/hyper/hyper_/08_hyper_see_8561347_env_humanoid_d.cri_False_m.std_True_a.s.mod_biased/config.json
[2023-08-23 18:08:03,029][17523] Overriding arg 'train_dir' with value 'train_dir/hyper/hyper_' passed from command line
[2023-08-23 18:08:03,030][17523] Adding new argument 'fps'=0 that is not in the saved config file!
[2023-08-23 18:08:03,030][17523] Adding new argument 'eval_env_frameskip'=None that is not in the saved config file!
[2023-08-23 18:08:03,031][17523] Adding new argument 'no_render'=False that is not in the saved config file!
[2023-08-23 18:08:03,031][17523] Adding new argument 'save_video'=False that is not in the saved config file!
[2023-08-23 18:08:03,032][17523] Adding new argument 'video_frames'=1000000000.0 that is not in the saved config file!
[2023-08-23 18:08:03,032][17523] Adding new argument 'video_name'=None that is not in the saved config file!
[2023-08-23 18:08:03,032][175

In [32]:
list_of_test_archs = [
                [4],                                   
            ]
actor_critic = create_actor_critic(cfg, env.observation_space, env.action_space)
actor_critic.eval()

device = torch.device("cpu" if cfg.device == "cpu" else "cuda")
actor_critic.model_to_device(device)

policy_id = cfg.policy_index
name_prefix = dict(latest="checkpoint", best="best")[cfg.load_checkpoint_kind]
checkpoints = Learner.get_checkpoints(Learner.checkpoint_dir(cfg, policy_id), f"{name_prefix}_*")


if milestone_name is None:
    checkpoint_dict = Learner.load_checkpoint(checkpoints, device)
    milestone_to_load = checkpoint_dict["model"]
else:
    milestone_dir = os.path.join(cfg.train_dir, cfg.experiment, "checkpoint_p0", "milestones")
    milestones = Learner.get_checkpoints(milestone_dir, "checkpoint_*")
    milestone_to_load = [milestone for milestone in milestones if milestone_name in milestone][0]
    checkpoint_dict = Learner.load_checkpoint([milestone_to_load], device)

actor_critic.load_state_dict(checkpoint_dict["model"])
ghn = actor_critic.actor_encoder.ghn
ghn.eval()    

# test_arch_model = MlpNetwork(fc_layers=arch_to_test, inp_dim = env.observation_space['obs'].shape[0], out_dim = env.action_space.shape[0]).to(device=device)
list_of_test_arch_indices = [[i for i,arc in enumerate(actor_critic.actor_encoder.list_of_arcs) if list(arc) == t_arc][0] for t_arc in list_of_test_archs]
list_of_test_shape_inds = torch.stack([actor_critic.actor_encoder.list_of_shape_inds[index][0:11] for k,index in enumerate(list_of_test_arch_indices)])

# arch_index = [i for i,arc in enumerate(actor_critic.actor_encoder.list_of_arcs) if list(arc) == arch_to_test][0]
# shape_ind = actor_critic.actor_encoder.list_of_shape_inds[arch_index]
# shape_ind = shape_ind[:torch.where(shape_ind == -1.0)[0][0]]

# _ = ghn([test_arch_model], return_embeddings=False, shape_ind = shape_ind.view(-1,1))
actor_critic.actor_encoder.set_graph(list_of_test_arch_indices, list_of_test_shape_inds)
test_arch_policy = actor_critic.actor_encoder.current_model[0]


[2023-08-23 18:13:25,266][17523] RunningMeanStd input shape: (240,)
[2023-08-23 18:13:25,302][17523] RunningMeanStd input shape: (1,)


AttributeError: 'AttrDict' object has no attribute 'milestone_name'

In [ ]:
i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
display(i)